In [1]:
import pandas as pd
import  math
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import re

## Pipeline for Matching Coverage

In [2]:
###############################################################################################################
##                                               PIPELINE                                                    ##
###############################################################################################################

In [2]:
'''
Clean the estimated haplotypes (haplotypes from haplomat=>drop ~ and g/G)
'''
def clean_data(
                filename, #dataframe with haplotypes and frequency,
                n, #number of samples that the haplotypes derived
              ):
    '''
    Read Estimated Haplotypes from Haplomat
    '''
    if 'DKMS' in filename and '3loci' in filename:
        hap_estimated = pd.read_csv(f'/Users/vasou/Documents/HLA/Matching Coverage/Estimated_Haplotypes/{filename}.dat',
                                    sep = '\t',
                                    decimal=',',
                                    header = None)
    else:
        hap_estimated = pd.read_csv(f'/Users/vasou/Documents/HLA/Matching Coverage/Estimated_Haplotypes/{filename}.dat',
                                    sep = '\t',
                                    header = None)
        
    #hap_estimated = pd.read_csv(f'/Users/vasou/Documents/GitHub/HLA.github.io/Analyses/Haplotype_Analyses/Haplotype_data/{filename}.dat', sep="\t",header=None)
    hap_estimated.columns = ['Haplotypes', 'Frequency']
    #display(hap_estimated)
    print(f'previous len: {hap_estimated.shape[0]}')
    correction = 1/(2*n)
    hap_estimated = hap_estimated[hap_estimated['Frequency']>=correction]
    
    #We want the estimated haplotpyes have >=2 counts (second correction)
    counts = [round(freq*n) for freq in hap_estimated['Frequency'].values]
   
    hap_estimated['Counts'] = counts
    counts_correction = 2
    #assert sum(counts) == n
    hap_estimated = hap_estimated[hap_estimated['Counts']>=counts_correction]
    print(f'samples: {sum(hap_estimated["Counts"].values)}')
    haplotypes = []
    for haplotype in hap_estimated['Haplotypes'].values:
        splited_haplotype = haplotype.split('~')
        splited_haplotype = [splited_haplotype[i][:-1] if splited_haplotype[i][-1].isalpha() else splited_haplotype[i] for i in range(len(splited_haplotype)) ]
        haplotype = ' '.join(splited_haplotype)
        haplotypes.append(haplotype)
    #display(hap_estimated)
    print(f'After len: {hap_estimated.shape[0]}')
    return haplotypes

In [3]:
'''
Calculate matching coverage
'''

def matching_coverage(
                        dat_data,
                        haplotypes, #estimated haplotypes from haplomat or haplotypes from homozygous
                        genes,
                        not_found = False
                    ):
    not_found_ids = []
    
    if not_found:
        if genes == 3:
            case = ['ID','A1', 'A2','B1','B2','DRB1_1', 'DRB1_2']
            for index, row in dat_data[case].iterrows():
                if row['A1'] == 'not found' and row['A2'] == 'not found':
                    not_found_ids.append(index)
                elif row['B1'] == 'not found' and row['B2'] == 'not found':
                    not_found_ids.append(index)
                elif row['DRB1_1'] == 'not found' and row['DRB1_2'] == 'not found':
                    not_found_ids.append(index)
        else:
            case = ['ID','A1', 'A2','B1','B2','C1','C2','DRB1_1', 'DRB1_2','DQB1_1', 'DQB1_2']
            for index, row in dat_data[case].iterrows():
                if row['A1'] == 'not found' and row['A2'] == 'not found':
                    not_found_ids.append(index)
                elif row['B1'] == 'not found' and row['B2'] == 'not found':
                    not_found_ids.append(index)
                elif row['C1'] == 'not found' and row['C2'] == 'not found':
                    not_found_ids.append(index)
                elif row['DRB1_1'] == 'not found' and row['DRB1_2'] == 'not found':
                    not_found_ids.append(index)
                elif row['DQB1_1'] == 'not found' and row['DQB1_2'] == 'not found':
                    not_found_ids.append(index)
    else:
        if genes == 3:
            case = ['ID','A1', 'A2','B1','B2','DRB1_1', 'DRB1_2']
        else:
            case = ['ID','A1', 'A2','B1','B2','C1','C2','DRB1_1', 'DRB1_2','DQB1_1', 'DQB1_2']
            
        for index, row in dat_data[case].iterrows():
                sum_row = sum([1 for allele in case[1:] if row[allele]=='not found'])
                if sum_row > 0:
                    not_found_ids.append(index)
        
        
    #data_nf = dat_data[case][~dat_data[case].apply(lambda row: row.astype(str).str.contains('not found').any(), axis=1)]
    dat_data.drop(labels = not_found_ids, axis = 0, inplace = True)
            
    #Add the loci before allele: A*01:01
    data = dat_data[case].copy()
    for column in data.columns[1:]: #the first column is the ID of each sample
        l = []
        for value in data[column].values:
            if column in ['DRB1_1', 'DRB1_2','DQB1_1','DQB1_2','DPB1_1','DPB1_2']:
                l.append(column.split('_')[0] + '*' + ':'.join(value.split(':')[:2]))
            else: 
                l.append(column[0] + '*' + ':'.join(value.split(':')[:2]))

        data[column] = l
    #data.to_excel('test.xlsx')  
    display(data)
    count_hap = defaultdict(lambda:0)
    ids = defaultdict(lambda:[])
    remaining_data = data.copy()
    for top in haplotypes:
        unique_hap = top.split(' ')
        
        for index, row in remaining_data.iterrows():
            haplotype = []
            if unique_hap[0] in row['A1'] or unique_hap[0] in row['A2']:
                haplotype.append(unique_hap[0])

            if unique_hap[1] in row['B1'] or unique_hap[1] in row['B2']:
                haplotype.append(unique_hap[1])
                
            if genes ==3:
                if unique_hap[2] in row['DRB1_1'] or unique_hap[2] in row['DRB1_2']:
                    haplotype.append(unique_hap[2])
                
            if genes == 5:
                if unique_hap[2] in row['C1'] or unique_hap[2] in row['C2']:
                    haplotype.append(unique_hap[2])
                if unique_hap[3] in row['DQB1_1'] or unique_hap[3] in row['DQB1_2']:
                    haplotype.append(unique_hap[3])
                if unique_hap[4] in row['DRB1_1'] or unique_hap[4] in row['DRB1_2']:
                    haplotype.append(unique_hap[4])  

            haplotype = ' '.join(haplotype)
            #print(haplotype)
            #print('-------------------------------------------')
            #print(top)
            #print('=======================================')
            if haplotype == top:
                count_hap[haplotype]+=1
                ids[haplotype].append((remaining_data['ID'][index],index))
                #print(haplotype)
        print(remaining_data.shape)
        remaining_data.drop(labels = [i[1] for i in ids[top]], axis = 0, inplace = True)
        print(remaining_data.shape)
        
    coverage = defaultdict()
    n = data.shape[0]#dat_data.shape[0]
    print(n)
    if n==0:
        n=0.0001
    for key in count_hap:
        coverage[key] = count_hap[key]/n
            
    return coverage, ids, count_hap

#matching_coverage,ids, count_hap = matching_coverage(data_4451, est_hap, genes = 3)

In [42]:

#cyprus_cbus_2_fields_3loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Cyprus/cyprus_cbus_2_fields_3loci.xlsx', header = [0])
#cyprus_cbus_2_fields_5loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Cyprus/cyprus_cbus_2_fields_5loci.xlsx', header = [0])

#cyprus_donors_2_fields_3loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Cyprus/cyprus_donors_2_fields_3loci.xlsx', header = [0])
#cyprus_donors_2_fields_5loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Cyprus/cyprus_donors_2_fields_5loci.xlsx', header = [0])

#cyprus_all_2_fields_3loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Cyprus/cyprus_cbus_plus_donors_2_fields_3loci.xlsx', header = [0])
#cyprus_all_2_fields_5loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Cyprus/cyprus_cbus_plus_donors_2_fields_5loci.xlsx', header = [0])

#greece_CBUs_2fields_3loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Greece/Greek_CBUS_2fields_3loci.xlsx', dtype = 'str', header = [0])
#greece_CBUs_2fields_5loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Greece/Greek_CBUS_2fields_5loci.xlsx', dtype = 'str', header = [0])

#greece_BMDs_2fields_3loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Greece/Greek_donors_2fields_3loci.xlsx', dtype = 'str', header = [0])
#greece_BMDs_2fields_5loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/Data/Greece/Greek_donors_2fields_5loci.xlsx', dtype = 'str', header = [0])

greece_CBUs_BMDs_2fields_3loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/MC_for_paper/Data/CBUs_BMDs_for_Hap_from_80804.xlsx', 
                                               sheet_name = 'CBUs&BMDs_2fields_3loci_78611',
                                               header = [0])
greece_CBUs_BMDs_2fields_5loci = pd.read_excel(f'/Users/vasou/Documents/HLA/Matching Coverage/MC_for_paper/Data/CBUs_BMDs_for_Hap_from_80804.xlsx',
                                               sheet_name = 'CBUs&BMDs_2fields_5loci_70222',
                                               header = [0])

#CYGR_cbus_2fields_3loci = pd.concat([greece_CBUs_2fields_3loci, cyprus_cbus_2_fields_3loci], ignore_index = True)
#CYGR_cbus_2fields_5loci = pd.concat([greece_CBUs_2fields_5loci, cyprus_cbus_2_fields_5loci], ignore_index = True)

#CYGR_BMDs_2fields_3loci = pd.concat([greece_BMDs_2fields_3loci, cyprus_donors_2_fields_3loci], ignore_index = True)
#CYGR_BMDs_2fields_5loci = pd.concat([greece_BMDs_2fields_5loci, cyprus_donors_2_fields_5loci], ignore_index = True)

#CYGR_all_2fields_3loci = pd.concat([greece_CBUs_BMDs_2fields_3loci, cyprus_all_2_fields_3loci], ignore_index = True)
#CYGR_all_2fields_5loci = pd.concat([greece_CBUs_BMDs_2fields_5loci, cyprus_all_2_fields_5loci], ignore_index = True)


#### 2 Groups:
    italia+turkey+Greece_DKMS+CYPRUS+Romania+bosnia+croatia (katw apo 50%)
    Austria, Holland, France, Portugal, Spain,ta ypoloipa (panw apo 50%)
#### ola mazi


In [37]:
60000 + 69130 + 73004

202134

In [35]:


Greece_group_1_5loci = ['Greece/RunX_htf_Greek_BMDs_2fields_69130_5loci',
                'DKMS/RunX_htf_Bosnia_10000_5loci', 'DKMS/RunX_htf_Croatia_10000_5loci', 'DKMS/RunX_htf_Greece_10000_5loci',
                'DKMS/RunX_htf_Italy_10000_5loci', 'DKMS/RunX_htf_Romania_10000_5loci', 'DKMS/RunX_htf_Turkey_10000_5loci',
                'Cyprus/RunX_htf_cyprus_donors_2fields_73004_5loci',
                ]
Greece_group_1_3loci = ['Greece/RunX_htf_Greek_BMDs_2fields_75599_3loci',
                'DKMS/RunX_htf_Bosnia_10000_3loci', 'DKMS/RunX_htf_Croatia_10000_3loci', 'DKMS/RunX_htf_Greece_10000_3loci',
                'DKMS/RunX_htf_Italy_10000_3loci', 'DKMS/RunX_htf_Romania_10000_3loci', 'DKMS/RunX_htf_Turkey_10000_3loci',
                'Cyprus/RunX_htf_cyprus_donors_2fields_96155_3loci',
                ]

Greece_group_2_5loci = ['Greece/RunX_htf_Greek_BMDs_2fields_69130_5loci',
                'DKMS/RunX_htf_Austria_10000_5loci', 'DKMS/RunX_htf_Dutch_10000_5loci', 'DKMS/RunX_htf_France_10000_5loci',
                'DKMS/RunX_htf_Portugal_10000_5loci', 'DKMS/RunX_htf_Spain_10000_5loci',
                ]


Greece_group_2_3loci = ['Greece/RunX_htf_Greek_BMDs_2fields_75599_3loci',
                'DKMS/RunX_htf_Austria_10000_3loci', 'DKMS/RunX_htf_Dutch_10000_3loci', 'DKMS/RunX_htf_France_10000_3loci',
                'DKMS/RunX_htf_Portugal_10000_3loci', 'DKMS/RunX_htf_Spain_10000_3loci',
                ]
#all_together = [
 #               'DKMS/RunX_htf_Austria_10000_3loci', 'DKMS/RunX_htf_Dutch_10000_3loci', 'DKMS/RunX_htf_France_10000_3loci',
  #              'DKMS/RunX_htf_Portugal_10000_3loci', 'DKMS/RunX_htf_Spain_10000_3loci',
   #             'DKMS/RunX_htf_Bosnia_10000_3loci', 'DKMS/RunX_htf_Croatia_10000_3loci', 'DKMS/RunX_htf_Greece_10000_3loci',
    #            'DKMS/RunX_htf_Italy_10000_3loci', 'DKMS/RunX_htf_Romania_10000_3loci', 'DKMS/RunX_htf_Turkey_10000_3loci',
     #           'Cyprus/RunX_htf_cyprus_donors_2fields_96155_3loci'
      #          ]
    
Greece_all_together_3 = ['Greece/RunX_htf_Greek_BMDs_2fields_75599_3loci',
                'DKMS/RunX_htf_Austria_10000_3loci', 'DKMS/RunX_htf_Dutch_10000_3loci', 'DKMS/RunX_htf_France_10000_3loci',
                'DKMS/RunX_htf_Portugal_10000_3loci', 'DKMS/RunX_htf_Spain_10000_3loci',
                'DKMS/RunX_htf_Bosnia_10000_3loci', 'DKMS/RunX_htf_Croatia_10000_3loci', 'DKMS/RunX_htf_Greece_10000_3loci',
                'DKMS/RunX_htf_Italy_10000_3loci', 'DKMS/RunX_htf_Romania_10000_3loci', 'DKMS/RunX_htf_Turkey_10000_3loci',
                'Cyprus/RunX_htf_cyprus_donors_2fields_96155_3loci'
                ]

Greece_all_together = ['Greece/RunX_htf_Greek_BMDs_2fields_69130_5loci',
                'DKMS/RunX_htf_Austria_10000_5loci', 'DKMS/RunX_htf_Dutch_10000_5loci', 'DKMS/RunX_htf_France_10000_5loci',
                'DKMS/RunX_htf_Portugal_10000_5loci', 'DKMS/RunX_htf_Spain_10000_5loci',
                'DKMS/RunX_htf_Bosnia_10000_5loci', 'DKMS/RunX_htf_Croatia_10000_5loci', 'DKMS/RunX_htf_Greece_10000_5loci',
                'DKMS/RunX_htf_Italy_10000_5loci', 'DKMS/RunX_htf_Romania_10000_5loci', 'DKMS/RunX_htf_Turkey_10000_5loci',
                'Cyprus/RunX_htf_cyprus_donors_2fields_73004_5loci'
                ]

group_1_haps = defaultdict(lambda:0)
for file in Greece_group_2_3loci:
    if '5loci' in file:
        hap_estimated = pd.read_csv(f'/Users/vasou/Documents/HLA/Matching Coverage/Estimated_Haplotypes/{file}.dat',
                                        sep = '\t',
                                    header = None,
                                   )
        
    elif not 'Cyprus' in file:
        hap_estimated = pd.read_csv(f'/Users/vasou/Documents/HLA/Matching Coverage/Estimated_Haplotypes/{file}.dat',
                                        sep = '\t',
                                        decimal=',',
                                        header = None,
                                   )
    else:
        hap_estimated = pd.read_csv(f'/Users/vasou/Documents/HLA/Matching Coverage/Estimated_Haplotypes/{file}.dat',
                                        sep = '\t',
                                    header = None,
                                   )
        

        
    hap_estimated.columns = ['Haplotypes', 'Frequency']
    for index, row in hap_estimated.iterrows():
        hap = ''.join(row['Haplotypes'].split('g')[:3])
        group_1_haps[hap] += round(float(row['Frequency'])*2*int(file.split('_')[-2]))
print(sum(group_1_haps.values()))        
a = {k: v / sum(group_1_haps.values()) for k, v in group_1_haps.items()}
print(sum(a.values()))
group_1 = pd.DataFrame(a, index = [0]).T
a
group_1.sort_values(by=[0], ascending = False, inplace = True)
group_1
group_1.to_csv('RunX_htf_Greece_Group2_125599_3loci.dat', sep ='\t')

249339
1.000000000000208


In [43]:
with pd.ExcelWriter('MC_2_2fields_Greece_12Countries_BMDs_to_Greece_all.xlsx', engine = "openpyxl", mode = "w") as writer:
    for n_gene in [3,5]:
        if n_gene == 5:
            #print('lalal')
            #est_hap = clean_data('Greece/RunX_htf_Greek_CBUs_2fields_1092_5loci', 1092)
            #est_hap = clean_data('Greece/RunX_htf_Greek_BMDs_2fields_69130_5loci', 69130)
            #est_hap = clean_data('Greece/RunX_htf_Greek_BMDs_CBUs_2fields_70222_5loci', 70222)
            #est_hap = clean_data('Cyprus/RunX_htf_cyprus_cbus_2fields_2702_5loci', 2702)
            #est_hap = clean_data('Cyprus/RunX_htf_cyprus_donors_2fields_73004_5loci', 73004)
            #est_hap = clean_data('Cyprus/RunX_htf_cyprus_cbus_plus_donors_2fields_75706_5loci', 75706)
            #est_hap = clean_data('RunX_htf_CBUs_CYGR_2fields_3794_5loci', 3794)
            #est_hap = clean_data('RunX_htf_CYGR_BMDs_2fields_142134_5loci', 142134)
            #est_hap = clean_data('RunX_htf_CBUs_BMDs_CYGR_2fields_145928_5loci', 145928)
            #est_hap = clean_data('Greece/RunX_htf_CreteAdults_1582_5loci', 1582)
            #est_hap = clean_data('DKMS/RunX_htf_Austria_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Bosnia_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Croatia_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Dutch_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_France_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Greece_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Italy_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Portugal_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Romania_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Spain_10000_5loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Turkey_10000_5loci', 10000)
            #est_hap = clean_data('RunX_htf_Group_1_133004_5loci', 133004)
            #est_hap = clean_data('RunX_htf_Group_2_50000_5loci', 50000)
            #est_hap = clean_data('RunX_htf_12_Countries_183004_5loci', 183004)
            #est_hap = clean_data('RunX_htf_Greece_Group1_202134_5loci', 202134)
            #est_hap = clean_data('RunX_htf_Greece_Group2_119130_5loci', 119130)
            est_hap = clean_data('RunX_htf_Greece_12Countries_252134_5loci', 252134)
            data = greece_CBUs_BMDs_2fields_5loci
            if 'BIRTH' in data.columns:
                data.columns = ['ID','BIRTH', 'A1', 'A2','B1','B2','C1','C2','DRB1_1','DRB1_2','DQB1_1','DQB1_2']
            else:
                data.columns = ['ID', 'A1', 'A2','B1','B2','C1','C2','DRB1_1','DRB1_2','DQB1_1','DQB1_2']
        else:
            #est_hap = clean_data('Greece/RunX_htf_Greek_CBUs_2fields_3012_3loci', 3012)
            #est_hap = clean_data('Greece/RunX_htf_Greek_BMDs_2fields_75599_3loci', 75599)
            #est_hap = clean_data('Greece/RunX_htf_Greek_BMDs_CBUs_2fields_78611_3loci', 78611)
            #est_hap = clean_data('Cyprus/RunX_htf_cyprus_cbus_2fields_2843_3loci', 2843)
            #est_hap = clean_data('Cyprus/RunX_htf_cyprus_donors_2fields_96155_3loci', 96155)
            #est_hap = clean_data('Cyprus/RunX_htf_cyprus_cbus_plus_donors_2fields__98998_3loci', 98998)
            #est_hap = clean_data('RunX_htf_CBUs_CYGR_2fields_5855_3loci', 5855)
            #est_hap = clean_data('RunX_htf_CYGR_BMDs_2fields_171754_3loci', 171754)
            #est_hap = clean_data('RunX_htf_CBUs_BMDs_CYGR_2fields_177609_3loci', 177609)
            #est_hap = clean_data('Greece/RunX_htf_CreteAdults_1582_3loci', 1582)
            #est_hap = clean_data('DKMS/RunX_htf_Austria_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Bosnia_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Croatia_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Dutch_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_France_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Greece_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Italy_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Portugal_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Romania_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Spain_10000_3loci', 10000)
            #est_hap = clean_data('DKMS/RunX_htf_Turkey_10000_3loci', 10000)
            #est_hap = clean_data('RunX_htf_Group_1_156155_3loci', 156155)
            #est_hap = clean_data('RunX_htf_Group_2_50000_3loci', 50000)
            #est_hap = clean_data('RunX_htf_12_Countries_206155_3loci', 206155)
            #est_hap = clean_data('RunX_htf_Greece_Group1_231754_3loci', 231754)
            #est_hap = clean_data('RunX_htf_Greece_Group2_125599_3loci', 125599)
            est_hap = clean_data('RunX_htf_Greece_12Countries_281754_3loci', 281754)
            data = greece_CBUs_BMDs_2fields_3loci
            if 'BIRTH' in data.columns:
                data.columns = ['ID','BIRTH', 'A1', 'A2','B1','B2','DRB1_1', 'DRB1_2']
            else:
                data.columns = ['ID','A1', 'A2','B1','B2','DRB1_1', 'DRB1_2']
                
            
        matching_cov, ids, count_hap = matching_coverage(data, est_hap, genes = n_gene, not_found = False)
        #not_found = True if allow not found
            
        a = pd.DataFrame(
            [list(count_hap.keys()), list(count_hap.values()), list(matching_cov.values())],
            ).T
        a.columns = ['Haplotype','Counts', 'Coverage']
        a.to_excel(writer, sheet_name = f'{n_gene}_genes', engine = "openpyxl", index = False)

previous len: 59379
samples: 275792
After len: 9753


,ID,A1,A2,B1,B2,DRB1_1,DRB1_2
0,HLA021S5,A*02:01,A*11:01,B*15:01,B*39:01,DRB1*12:01,DRB1*16:01
1,HLA021S6,A*02:01,A*11:01,B*35:01,B*51:01,DRB1*01:01,DRB1*13:01
2,HLA021S7,A*23:01,A*24:02,B*08:01,B*41:01,DRB1*03:01,DRB1*04:05
3,HLA021S8,A*01:01,A*02:01,B*15:24,B*57:01,DRB1*07:01,DRB1*15:01
4,HLA021S10,A*30:04,A*31:01,B*35:01,B*49:01,DRB1*01:01,DRB1*13:02
...,...,...,...,...,...,...,...
78606,CBMDP-0083085,A*02:01,A*32:01,B*14:02,B*51:01,DRB1*01:02,DRB1*04:02
78607,CBMDP-0083086,A*24:02,A*33:01,B*14:02,B*18:01,DRB1*04:04,DRB1*11:04
78608,CBMDP-0083087,A*23:01,A*33:03,B*35:08,B*44:03,DRB1*07:01,DRB1*13:03
78609,CBMDP-0083891,A*02:01,A*68:02,B*18:05,B*53:01,DRB1*13:02,DRB1*16:01


(78611, 7)
(75077, 7)
(75077, 7)
(69122, 7)
(69122, 7)
(67933, 7)
(67933, 7)
(67151, 7)
(67151, 7)
(65453, 7)
(65453, 7)
(63807, 7)
(63807, 7)
(62879, 7)
(62879, 7)
(59998, 7)
(59998, 7)
(58011, 7)
(58011, 7)
(57213, 7)
(57213, 7)
(56600, 7)
(56600, 7)
(55830, 7)
(55830, 7)
(55189, 7)
(55189, 7)
(54613, 7)
(54613, 7)
(53864, 7)
(53864, 7)
(53037, 7)
(53037, 7)
(52851, 7)
(52851, 7)
(51886, 7)
(51886, 7)
(51457, 7)
(51457, 7)
(50788, 7)
(50788, 7)
(50386, 7)
(50386, 7)
(49794, 7)
(49794, 7)
(48960, 7)
(48960, 7)
(48396, 7)
(48396, 7)
(48100, 7)
(48100, 7)
(47949, 7)
(47949, 7)
(47290, 7)
(47290, 7)
(46662, 7)
(46662, 7)
(45472, 7)
(45472, 7)
(45153, 7)
(45153, 7)
(44747, 7)
(44747, 7)
(44160, 7)
(44160, 7)
(44081, 7)
(44081, 7)
(43828, 7)
(43828, 7)
(43317, 7)
(43317, 7)
(42837, 7)
(42837, 7)
(42349, 7)
(42349, 7)
(42011, 7)
(42011, 7)
(41907, 7)
(41907, 7)
(41591, 7)
(41591, 7)
(40797, 7)
(40797, 7)
(40374, 7)
(40374, 7)
(40157, 7)
(40157, 7)
(40014, 7)
(40014, 7)
(39729, 7)
(39729, 7)

(10681, 7)
(10649, 7)
(10649, 7)
(10637, 7)
(10637, 7)
(10603, 7)
(10603, 7)
(10578, 7)
(10578, 7)
(10524, 7)
(10524, 7)
(10522, 7)
(10522, 7)
(10506, 7)
(10506, 7)
(10454, 7)
(10454, 7)
(10435, 7)
(10435, 7)
(10388, 7)
(10388, 7)
(10364, 7)
(10364, 7)
(10340, 7)
(10340, 7)
(10314, 7)
(10314, 7)
(10292, 7)
(10292, 7)
(10286, 7)
(10286, 7)
(10246, 7)
(10246, 7)
(10235, 7)
(10235, 7)
(10202, 7)
(10202, 7)
(10198, 7)
(10198, 7)
(10155, 7)
(10155, 7)
(10143, 7)
(10143, 7)
(10089, 7)
(10089, 7)
(10079, 7)
(10079, 7)
(10071, 7)
(10071, 7)
(10049, 7)
(10049, 7)
(10013, 7)
(10013, 7)
(10001, 7)
(10001, 7)
(9996, 7)
(9996, 7)
(9982, 7)
(9982, 7)
(9946, 7)
(9946, 7)
(9927, 7)
(9927, 7)
(9908, 7)
(9908, 7)
(9896, 7)
(9896, 7)
(9864, 7)
(9864, 7)
(9837, 7)
(9837, 7)
(9813, 7)
(9813, 7)
(9792, 7)
(9792, 7)
(9768, 7)
(9768, 7)
(9747, 7)
(9747, 7)
(9705, 7)
(9705, 7)
(9683, 7)
(9683, 7)
(9640, 7)
(9640, 7)
(9612, 7)
(9612, 7)
(9600, 7)
(9600, 7)
(9594, 7)
(9594, 7)
(9536, 7)
(9536, 7)
(9505, 7)
(9505

(4659, 7)
(4655, 7)
(4655, 7)
(4636, 7)
(4636, 7)
(4635, 7)
(4635, 7)
(4624, 7)
(4624, 7)
(4624, 7)
(4624, 7)
(4618, 7)
(4618, 7)
(4614, 7)
(4614, 7)
(4607, 7)
(4607, 7)
(4592, 7)
(4592, 7)
(4588, 7)
(4588, 7)
(4576, 7)
(4576, 7)
(4564, 7)
(4564, 7)
(4549, 7)
(4549, 7)
(4544, 7)
(4544, 7)
(4533, 7)
(4533, 7)
(4533, 7)
(4533, 7)
(4527, 7)
(4527, 7)
(4522, 7)
(4522, 7)
(4519, 7)
(4519, 7)
(4517, 7)
(4517, 7)
(4495, 7)
(4495, 7)
(4454, 7)
(4454, 7)
(4450, 7)
(4450, 7)
(4439, 7)
(4439, 7)
(4435, 7)
(4435, 7)
(4427, 7)
(4427, 7)
(4420, 7)
(4420, 7)
(4415, 7)
(4415, 7)
(4412, 7)
(4412, 7)
(4411, 7)
(4411, 7)
(4399, 7)
(4399, 7)
(4394, 7)
(4394, 7)
(4382, 7)
(4382, 7)
(4351, 7)
(4351, 7)
(4344, 7)
(4344, 7)
(4332, 7)
(4332, 7)
(4314, 7)
(4314, 7)
(4308, 7)
(4308, 7)
(4294, 7)
(4294, 7)
(4282, 7)
(4282, 7)
(4270, 7)
(4270, 7)
(4269, 7)
(4269, 7)
(4264, 7)
(4264, 7)
(4262, 7)
(4262, 7)
(4259, 7)
(4259, 7)
(4251, 7)
(4251, 7)
(4250, 7)
(4250, 7)
(4248, 7)
(4248, 7)
(4248, 7)
(4248, 7)
(4227, 7)


...

(226, 11)
(226, 11)
(226, 11)
(226, 11)
(226, 11)
(226, 11)
(226, 11)
(226, 11)
(226, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(225, 11)
(224, 11)
(224, 11)
(224, 11)
(224, 11)
(224, 11)
(224, 11)
(224, 11)
(224, 11)
(224, 11)
(224, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)
(223, 11)


(195, 11)
(195, 11)
(195, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(194, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(193, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)
(192, 11)


(163, 11)
(163, 11)
(163, 11)
(163, 11)
(163, 11)
(163, 11)
(163, 11)
(163, 11)
(163, 11)
(163, 11)
(163, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(162, 11)
(161, 11)
(161, 11)
(160, 11)
(160, 11)
(159, 11)
(159, 11)
(159, 11)
(159, 11)
(158, 11)
(158, 11)
(158, 11)
(158, 11)
(158, 11)
(158, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(157, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)
(156, 11)


(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)
(142, 11)


(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(132, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)
(131, 11)


- Austria 
    5loci
    - previous len: 17790  
    - samples: 7230  
    - After len: 1046  
    3loci
    - previous len: 8404
    - samples: 8021
    - After len: 1039
    

- Bosnia  
    5loci
    - previous len: 11617  
    - samples: 7935  
    - After len: 1059
    3loci
    - previous len: 6037
    - samples: 8525
    - After len: 1005
    
- Croatia 
    5loci
    - previous len: 15044  
    - samples: 7799  
    - After len: 1087
    3loci
    - previous len: 7675
    - samples: 8429
    - After len: 1036
    
- Dutch 
    5loci
    - previous len: 19994  
    - samples: 7550  
    - After len: 977
    3loci
    - previous len: 9507
    - samples: 8257
    - After len: 946
    
- France 
    5loci
    - previous len: 18971  
    - samples: 6786  
    - After len: 1009
    3loci
    - previous len: 9699
    - samples: 7661
    - After len: 1043
    
- GreeceDKMS 
    5loci
    - previous len: 13696  
    - samples: 7197  
    - After len: 1228
    3loci
    - previous len: 7654
    - samples: 7989
    - After len: 1180
    
- Italy  
    5loci
    - previous len: 17403  
    - samples: 6587  
    - After len: 1149
    3loci
    - previous len: 9381
    - samples: 7413
    - After len: 1184
    
- Portugal  
    5loci
    - previous len: 16167  
    - samples: 7123  
    - After len: 1175
    3loci
    - previous len: 8434
    - samples: 7894
    - After len: 1164
    
- Romania
    5loci
    - previous len: 13178  
    - samples: 7329  
    - After len: 1136
    3loci
    - previous len: 7271
    - samples: 8027
    - After len: 1110
    
- Spain
    5loci
    - previous len: 15638
    - samples: 6930
    - After len: 1152
    3loci
    - previous len: 8232
    - samples: 7728
    - After len: 1189

- Turkey
    5loci
    - previous len: 21608
    - samples: 6436
    - After len: 1233
    3loci
    - previous len: 11179
    - samples: 7364
    - After len: 1253
    



- Cyprus CBUs 
    5loci/2fields
    - previous len: 1727
    - samples: 1971
    - After len: 465
    3loci/2fields
    - previous len: 1539
    - samples: 2172
    - After len: 470   
    
- Cyprus Donors 
    5loci/2fields
    - previous len: 9887
    - samples: 69126
    - After len: 3341 
    3loci/2fields  
    - previous len: 7904
    - samples: 93327
    - After len: 3335

- Cyprus All 
    5loci/2fields
    - previous len: 10100
    - samples: 71741
    - After len: 3405

    3loci/2fields  
    - previous len: 7976
    - samples: 96145
    - After len: 3371






- Greece CBUs
    5loci/2fields
    - previous len: 1133
    - samples: 559
    - After len: 185
    3loci/2fields  
    - previous len: 1977
    - samples: 2091
    - After len: 463
    
- Greece Donors
    5loci/2fields
    - previous len: 14203
    - samples: 63590
    - After len: 4982
    3loci/2fields  
    - previous len: 9354
    - samples: 72323
    - After len: 4184


- Greece All
    5loci/2fields
    - previous len: 14307
    - samples: 64679
    - After len: 5056
    3loci/2fields  
    - previous len: 9572
    - samples: 75277
    - After len: 4282





- Greece&Cyprus CBUs
    5loci/2fields
    - previous len: 2498
    - samples: 2685
    - After len: 605
    3loci/2fields  
    - previous len: 2885
    - samples: 4541
    - After len: 832
    
    
- Greece&Cyprus BMDs
    5loci/2fields
    - previous len: 18466
    - samples: 135077
    - After len: 6886
    3loci/2fields  
    - previous len: 12121
    - samples: 167677
    - After len: 5741


- Greece&Cyprus CBUs&BMDs
    5loci/2fields
    - previous len: 18652
    - samples: 138837
    - After len: 6985
    3loci/2fields  
    - previous len: 12264
    - samples: 173467
    - After len: 5831
    
    
    
    
    
- Group_1 BMDs
    5loci/2fields
    - previous len: 79399
    - samples: 124384
    - After len: 7344
    3loci/2fields
    - previous len: 36108
    - samples: 151814
    - After len: 6349 
    
- Group_2 BMDs
    5loci/2fields 
    - previous len: 72958
    - samples: 43162
    - After len: 4398
    3loci/2fields
    - previous len: 31209
    - samples: 46327
    - After len: 3933  

- Group_12_Countries BMDs
    5loci/2fields
    - previous len: 138367
    - samples: 170462
    - After len: 10035
    
    3loci/2fields
    - previous len: 56942
    - samples: 200387
    - After len: 8258
   
   
   
- Greece&Group_1
    5loci/2fields
    - previous len: 85940
    - samples: 192167
    - After len: 10208
    3loci/2fields
    - previous len: 39079
    - samples: 227176
    - After len: 8133 
    
- Greece&Group_2
    5loci/2fields
    - previous len: 82180
    - samples: 110323
    - After len: 8440
    3loci/2fields
    - previous len: 35875
    - samples: 121470
    - After len: 6725 
    
- Greece&Group_12_Countries
    5loci/2fields
    - previous len: 144108
    - samples: 238509
    - After len: 12636
    3loci/2fields
    - previous len: 59379
    - samples: 275792
    - After len: 9753
